http://neurotycho.org/expdatalist/listview?task=78の
20120730PF_Anesthesia+and+Sleep_Chibi_Toru+Yanagawa_mat_ECoG128
を
$share/ECoGなどと名付けたフォルダ内にダウンロードする。

In [ ]:
#ダウンロードしたファイルをtrainとdevとtestに分ける
import scipy.io
import csv
import pandas as pd
import numpy as np
import scipy.io
import os
import random
import pandas as pd
import torch
import torchaudio
import csv
dataroot='$share/ECoG/20120730PF_Anesthesia+and+Sleep_Chibi_Toru+Yanagawa_mat_ECoG128/Session1/'
root='$share/ECoG_training/Chibi_20120730_session1_CommonAve_ReginLabel/'
test_list=[ 24, 62, 3, 49, 11,  34, 54, 25, 53, 42, 50, 10, 8]
train_list=[7, 0, 45, 29, 36, 38, 66, 60, 40, 13, 2, 4, 39, 41, 51, 18, 44, 1, 52,  33, 17, 46, 26, 19,  61, 55, 16, 57, 63, 23, 14, 5, 67,  27, 65, 32, 48, 21, 31, 64, 30, 20, 56, 9, 22,15, 28, 47, 43, 37, 58, 35, 59, 6, 12]

region=[
    [5,6,14,15,23,24,25,26,34,35,36,37,45,46,47,48,49,105],#LP
    [7,16,17,27,28,38,39,50,51,55,62,92,93,94,107],#PM
    [8,9,10,18,19,20,29,30,31,40,41,42,43,95,108,125,126,127,128],#MS
    [1,2,3,4,11,12,21,22,33,32,44,121,122,123,124],#PC
    [65,66,67,68,69,70,71,80,81,82,83,96,97,98],#TC
    [13,72,73,84,85,86,87,99,100,109,110],#HV
    [74,75,76,77,78,79,88,89,90,91,101,102,103,104,111,112,113,114,115,116,117,118,119,120]
]
electrodes=[region[i][j] for i in range(len(region)) for j in range(len(region[i])) ]
num_electrodes=len(electrodes)
#print(electrodes,num_electrodes)
path=dataroot+'ECoG_ch1'+'.mat'
data_ECoG_ch= scipy.io.loadmat(path)
data=data_ECoG_ch['ECoGData_ch1'][0]
len_data=len(data)
lists_ECoG_CommonAve = np.array([0 for i in range(len_data)])
#num_CommonAve=np.array([i+1 for i in range(num_electrodes)])

for k in electrodes:
    num=k
    path=dataroot+'ECoG_ch'+str(num)+'.mat'
    data_ECoG_ch= scipy.io.loadmat(path)
    arg='ECoGData_ch'+str(num)    
    data=data_ECoG_ch[arg][0]
    print('data'+str(k),data)
    data=np.array(data[:2500000])
    lists_ECoG_CommonAve +=data
    #print('lists_ECoG_CommonAve'+str(k),lists_ECoG_CommonAve)
lists_ECoG_CommonAve=lists_ECoG_CommonAve/num_electrodes
#print('lists_ECoG_CommonAve',lists_ECoG_CommonAve)

for k in electrodes:
    num=k
    path=dataroot+'ECoG_ch'+str(num)+'.mat'
    data_ECoG_ch= scipy.io.loadmat(path)
    arg='ECoGData_ch'+str(num)    
    data=data_ECoG_ch[arg][0]
    data=np.array(data[:2500000])
    data=data-lists_ECoG_CommonAve
    #print('lists_ECoG_CommonAve',lists_ECoG_CommonAve)
    #print('data'+str(k),data)
    for j in train_list:
        for i in range(len(region)):
            if k in region[i]:
                outpath=root+'train/'+str(i)+'/'+str(i)+'-'+str(k)+'-'+str(j)
                print('outpath',outpath)
        if not os.path.isdir(outpath):
            os.makedirs(outpath)
        out_data=data[j*30000:j*30000+30000]
        out_data = torch.tensor(out_data)
        print('out_data',out_data)
        torch.save(out_data, outpath+'.pt')         
    for j in test_list:
        for i in range(len(region)):
            if k in region[i]:
                outpath=root+'test/'+str(i)+'/'+str(i)+'-'+str(k)+'-'+str(j)
                print('outpath',outpath)
        if not os.path.isdir(outpath):
            os.makedirs(outpath)
        out_data=data[j*30000:j*30000+30000]
        out_data = torch.tensor(out_data)
        print('out_data',out_data)
        torch.save(out_data, outpath+'.pt')

        

python $share/preprocess/generate_len_for_bucket_ECoG_preprocess_pretrain.py
を修正した後ターミナル上で実行
#ダウンロードしたファイルをtrainとdevとtestに分けた後、bucketfileを作る。

$share/pretrain/tera/config_runner.yamlのrootを変更

refine/s3prl/pretrain/tera/config_runner.yamlのrootを変更

python run_pretrain.py -u tera -g pretrain/tera/config_model.yaml -n '保存フォルダ名'
を実行